# Ingest SEC DERA NOTES data into Trino pipeline

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Run these in a notebook cell if you need to install onto your nb env

```python
# 'capture' magic prevents long outputs from spamming your notebook
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
```

In [1]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib

Load Environment Variables

In [7]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [8]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [9]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

Open a Trino connection using JWT for authentication

In [10]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'

In [11]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('default',)
('demo_dv',)
('iceberg_demo',)
('information_schema',)
('pcaf_sovereign_footprint',)
('sandbox',)


Enter the Pandas!

In [12]:
import pandas as pd
import re

Drop previous tables and schema to start with a fresh slate

Load `ticker` file (updated sporadically from https://www.sec.gov/include/ticker.txt)

In [13]:
ticker_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'SEC-DERA/ticker.txt')
ticker_file.download_file(f'/tmp/dera-ticker.txt')
ticker_df = pd.read_csv(f'/tmp/dera-ticker.txt', names=['tname', 'cik'], header=None, sep='\t', dtype={'tname':'string','cik':'int64'}, engine='c')
ticker_dict = dict(zip(ticker_df.cik, ticker_df.tname))

ticker_df

,tname,cik
0,aapl,320193
1,msft,789019
2,goog,1652044
3,amzn,1018724
4,tsla,1318605
...,...,...
12416,afacu,1849489
12417,afaqw,1841661
12418,aesc,874761
12419,aesew,1708341


Prepare GLEIF matching data

In [14]:
gleif_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'mtiemann-GLEIF/DERA-matches.csv')
gleif_file.download_file(f'/tmp/dera-gleif.csv')
gleif_df = pd.read_csv(f'/tmp/dera-gleif.csv', header=0, sep=',', dtype=str, engine='c')
gleif_dict = dict(zip(gleif_df.name, gleif_df.LEI))

Load the SUB, NUM, and TAG tables into Trino

In [106]:
# import re
import uuid

# Borrowed/stole this definition from SEC Corp Financials notebook...
float_tags = [
    'EntityPublicFloat',
    'StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest',
    'FreeFloat',
    'PublicFloat',
    'PublicFloatValue',
]

# These are in priority preference order
share_tags = [
    'EntityCommonStockSharesOutstanding',
    'CommonStockSharesOutstanding',
    'SharesOutstanding',
    'WeightedAverageNumberOfDilutedSharesOutstanding',
    'WeightedAverageNumberOfSharesOutstandingBasic',
]

# We don't want anything that reports par value, such as CommonStockValue
shareprice_tags = [
    'SharePrice',
    'PerSharePrice',
    'MarketValuePerShare',
    'SaleOfStockPricePerShare',
    'CashPricePerOrdinaryShare',
    'TreasurySharesValuePerShare',
    'SharesOutstandingPricePerShare',
]

treasury_share_tags = [
    'TreasuryStockShares',
    'TreasuryStockShares1',
]

treasury_value_tags = [
    'TreasurySharesMarketValue',
    'FairValueOfTreasuryShares',
    'MarketValueOfTreasuryShares',
]

all_float_helper_tags = share_tags + shareprice_tags + treasury_share_tags + treasury_value_tags

dera_regex = re.compile(r' ?/.*$')
dera_df = {}

from math import floor

def generate_intermediate_ddate(df_orig):
    if len(df_orig)==1:
        return df_orig
    year1 = df_orig.iloc[0].ddate
    year2 = df_orig.iloc[1].ddate
    if (year1-year2).days > 731:
        print("gap years")
        print(df_orig.iloc[0:2])
    new_df = df_orig.iloc[[0]].copy()
    year_end = pd.to_datetime(f"{floor((year1.year+year2.year)/2.0)}1231", format='%Y%m%d', utc=True)
    new_df.ddate = year_end
    if year1.year==year2.year:
        print("same years")
        print(df_orig.iloc[0:2])
        new_df.value = (df_orig.iloc[0].value + df_orig.iloc[1].value)/2.0
    else:
        new_df.value = ((365.0-(year1-year_end).days)*df_orig.iloc[0].value + (year_end-year2).days*df_orig.iloc[1].value)/365.0
    new_df.version = df_orig.iloc[0].adsh
    print(f"adding fact ({new_df.tag})")
    return new_df

# When this function is called, we already know that we have no matches in FLOAT_TAGS.
# GROUPED_DF is grouped by ADSH and only for annual reports.  DDATE can be anything (because many reports look back 1-5 years)
# We are working these annual reports quarter by quarter for the quarter in which they are reported

def infer_float(grouped_df):
    new_df = pd.DataFrame()
    for key, item in grouped_df:
        df = grouped_df.get_group(key)
        # We have no overall float value.  Build from shares outstanding * price
        df_shares = df[df.tag.isin(share_tags) & (df.value>0)]
        if df_shares.empty:
            continue
        else:
            for share_tag in share_tags:
                if not df_shares[df_shares.tag==share_tag].empty:
                    df_shares = df_shares[df_shares.tag==share_tag]
                    break
            if len(df_shares[df_shares.ddate.dt.year==df_shares.fy.dt.year])>1:
                print("thinning shares")
                df_shares = df_shares[df_shares.ddate.dt.year==df_shares.fy.dt.year]
                df_shares = df_shares.sort_values('ddate', ascending=False)
        df_prices = df[df.tag.isin(shareprice_tags)]
        # We have no overall price.  Build from price derived from treasury valuation
        if df_prices.empty:
            df_treasury_shares = df[df.tag.isin(treasury_share_tags)]
            df_treasury_value = df[df.tag.isin(treasury_value_tags)]
            if df_treasury_shares.empty or df_treasury_value.empty:
                continue
            df_svp = df_treasury_value.merge(df_treasury_shares, on=['adsh', 'ddate', 'coreg'])
            if df_svp.empty:
                print(f"{df.adsh.iat[0]}: merge failed (1)")
                continue
            # Pick latest date / largest number of shares as basis
            df_float = df_svp.sort_values(['ddate', 'value_y'], ascending=False).iloc[[0]].copy()
            price_per_share = df_float.value_x.squeeze() / df_float.value_y.squeeze()
            df_float.rename(columns={'uom_x':'uom'},inplace=True)
            tag = 'ComputedTreasuryFloat'
        else:
            # if df_prices[df_prices.tag.str.startswith('ShareBasedCompensationArrangementByShareBasedPaymentAward')].empty:
            #     print(f"Must use market prices; len(df_prices) =  {len(df_prices)}")
            # else:
            #     print(f"Can use Share Based Comp {df_prices.tag.str[45:]}:\n{df_prices}\n\n")
            # We derive a price from market reports
            df_svp = df_prices.merge(df_shares, on=['adsh', 'ddate', 'coreg'])
            if df_svp.empty:
                if len (df_prices[df_prices.ddate.dt.year==df_prices.fy.dt.year])>1:
                    print("thinning prices")
                    df_prices = df_prices[df_prices.ddate.dt.year==df_prices.fy.dt.year]
                if len(df_prices)<3 and len(df_shares)<3:
                    df_prices = generate_intermediate_ddate(df_prices.sort_values('ddate', ascending=False))
                    df_float = generate_intermediate_ddate(df_shares.sort_values('ddate', ascending=False))
                    price_per_share = df_prices.value.squeeze()
                    # print("merge rescued (2)")
                    # display(df_shares)
                else:
                    print(f"{df.adsh.iat[0]}: merge failed (2)")
                    print(f"len(df_prices) = {len(df_prices)}")
                    print(f"len(df_shares) = {len(df_shares)}")
                    display(df_prices)
                    display(df_shares)
                    continue
            else:
                # Pick latest date / largest number of shares as basis
                df_float = df_svp.sort_values(['ddate', 'value_y'], ascending=False).iloc[[0]]
                price_per_share = df_float.value_x.squeeze() # value_x is a price in this case
                df_float.rename(columns={'uom_x':'uom'},inplace=True)
            tag = 'ComputedMarketFloat'
        df_float = df_float[['adsh', 'ddate', 'uom', 'coreg']]
        df_float['tag'] = tag
        # TODO: should connect price ddate with total shares ddate
        total_shares = df_shares.iloc[0].value
        df_float['value'] = price_per_share * total_shares
        df_float['qtrs'] = 0
        df_float['srcdir'] = 'computed'
        df_float['version'] = df_float['footnote'] = pd.NA
        df_float['dimh'] = '0x00000000'
        df_float['iprx'] = 0
        df_float['dcml'] = 32767
        df_float['dimn'] = 0
        df_float['footlen'] = 0
        # These are not correct calculations
        df_float['durp'] = df_float['datp'] = 0
        df_float = df_float.astype(df.drop(columns=['fy','fp']).dtypes.to_dict())
        new_df = new_df.append(df_float)
    return new_df

def read_dera_notes_table(zf, fy_month_or_qtr, tbl):
    """From a local file ZF, read data for the period FY_QTR for the DERA table TBL.
    Return the Dataframe created so that when it is time to create the actual Trino table
    we know what the shape of the data should look like.  The returned DF has all the data
    of the specific ingestion, not all the data of all the ingestions of data for TBL."""
    global dera_df
    
    df = pd.read_csv(zf, header=0, sep='\t', dtype='string', keep_default_na=False, nrows = None, engine='c')
    df['srcdir'] = fy_qtr
    df.srcdir = df.srcdir.astype('string')
    
    # df = df.convert_dtypes (infer_objects=False, convert_string=True, convert_integer=False, convert_boolean=False, convert_floating=False)
    # Print the output
    # print(df.dtypes)
    
    if tbl=='sub':
        df.name = df.name.map(lambda x: re.sub(dera_regex, '', x)).astype('string')
        df['LEI'] = df.name.map(gleif_dict).astype('string')
        df.cik = df.cik.astype('int64')
        df.loc[df.sic=='', 'sic'] = pd.NA
        df.sic = df.sic.astype('Int16')
        df.loc[df.ein=='', 'ein'] = pd.NA
        df.ein = df.ein.astype('Int64')
        df.wksi = df.wksi.astype('bool')
        df.period = pd.to_datetime(df.period, format='%Y%m%d', utc=True, errors='coerce')
        df.fy = pd.to_datetime(df.fy, format='%Y', utc=True, errors='coerce')
        df.filed = pd.to_datetime(df.filed, format='%Y%m%d', utc=True)
        df.accepted = pd.to_datetime(df.accepted, format='%Y-%m-%d %H:%M:%S', utc=True)
        df.prevrpt = df.prevrpt.astype('bool')
        df.detail = df.detail.astype('bool')
        df.nciks = df.nciks.astype('int16')
        df.loc[df.pubfloatusd=='', 'pubfloatusd'] = pd.NA
        df.pubfloatusd = df.pubfloatusd.astype('Float32')
        df.floatdate = pd.to_datetime(df.floatdate, format='%Y%m%d', utc=True, errors='coerce')
        df.loc[df.floatmems=='', 'floatmems'] = pd.NA
        df.floatmems = df.floatmems.astype('Int16')

        cols = df.columns.tolist()
        # Move LEI to a more friendly location in the column order
        cols = cols[0:3] + [cols[-1]] + cols[3:-1]
        df = df[cols]
    elif tbl=='num':
        # documentation wrongly lists coreg as NUMERIC length 256.  It is ALPHANUMERIC.
        if fy_qtr=='2021q3':
            df.loc[df.ddate=='30210630', 'ddate'] = '20210630'
        if fy_qtr=='2019q2':
            df.loc[df.ddate=='29171231', 'ddate'] = '20171231'
        if fy_qtr=='2017q3':
            df.loc[df.ddate=='60160630', 'ddate'] = '20160630'
        # Fix some bad AES data
        if fy_qtr=='2021q1':
            df.loc[(df.adsh=='0000874761-21-000015')&(df.tag=='CommonStockValue')&(df.ddate=='20190630'), 'ddate'] = '20200630'
        elif fy_qtr=='2020q1':
            df.loc[(df.adsh=='0000874761-20-000012')&(df.tag=='EntityPublicFloat')&(df.ddate=='20180630'), 'ddate'] = '20190630'
            df.loc[(df.adsh=='0000874761-20-000012')&(df.tag=='CommonStockValue')&(df.ddate=='20180630'), 'ddate'] = '20190630'
        df.ddate = pd.to_datetime(df.ddate, format='%Y%m%d', utc=True)
        df.qtrs = df.qtrs.astype('int16')
        df.iprx = df.iprx.astype('int16')
        df.loc[df.coreg=='', 'coreg'] = pd.NA
        df.loc[df.value=='', 'value'] = pd.NA
        df.value = df.value.astype('Float64')
        df.loc[df.footnote=='', 'footnote'] = pd.NA
        df.footlen = df.footlen.astype('int16')
        df.dimn = df.dimn.astype('int16')
        df.durp = df.durp.astype('Float32')
        df.datp = df.datp.astype('Float32')
        df.dcml = df.dcml.astype('int16')
        
        print(f"Inferring floats: start len(df) = {len(df)}")
        annual_df = dera_df['sub'][dera_df['sub'].form.isin(['10-K','20-F','40-F'])]
        df['fy'] = df.adsh.map(dict(zip(annual_df.adsh,annual_df.fy)))
        df['fp'] = df.adsh.map(dict(zip(annual_df.adsh,annual_df.fp)))
        print(f"len(df[df.fp=='FY']) = {len(df[df.fp=='FY'])}")
        # df = df[df.fp=='FY']
        # df = df.assign(cik=df.adsh.str[:10])
        df_has_float = df[df.tag.isin(float_tags)]
        print(f"len(df_has_float) = {len(df_has_float)}")
        df_needs_float = df[~df.adsh.isin(df_has_float.adsh)]
        print(f"len(df_needs_float) = {len(df_needs_float)}")
        float_df = infer_float(df_needs_float[df_needs_float.coreg.isna()
                                              &df_needs_float.tag.isin(all_float_helper_tags)
                                              &(df_needs_float.value>0)].groupby(['adsh'], as_index=False))
        df = df.drop(columns=['fy','fp'])
        # print(df.dtypes)
        # print(float_df.dtypes)
        if float_df.empty:
            print(f"{len(float_df)} floats inferred; Sorry!")
        else:
            float_df = float_df.astype(df.dtypes.to_dict())
            df = pd.concat([df, float_df])
            print(f"{len(float_df)} floats inferred; {len(float_df[float_df.tag=='ComputedTreasuryFloat'])} treasury-based; {len(float_df[float_df.tag=='ComputedMarketFloat'])} market-based")
    elif tbl=='tag':
        df.custom = df.custom.astype('bool')
        df.abstract = df.abstract.astype('bool')
        df.loc[df.crdr=='', 'crdr'] = pd.NA
        df.loc[df.tlabel=='', 'tlabel'] = pd.NA
        df.loc[df.doc=='', 'doc'] = pd.NA
    elif tbl=='dim':
        df.rename(columns={'dimhash':'dimh'},inplace=True)
        df.segt = df.segt.astype('bool')
    elif tbl=='txt':
        df.ddate = pd.to_datetime(df.ddate, format='%Y%m%d', utc=True)
        df.qtrs = df.qtrs.astype('int32')
        df.iprx = df.iprx.astype('int16')
        df.dcml = df.dcml.astype('int16')
        df.durp = df.durp.astype('Float32')
        df.datp = df.datp.astype('Float32')
        df.dimn = df.dimn.astype('Int16')
        df.escaped = df.escaped.astype('bool')
        df.srclen = df.srclen.astype('int16')
        df.txtlen = df.txtlen.astype('int16')
        df.footlen = df.footlen.astype('int16')
    elif tbl=='ren':
        df.report = df.report.astype('int32')
        df.loc[df.parentreport=='', 'parentreport'] = pd.NA
        df.parentreport = df.parentreport.astype('Int32')
        df.loc[df.ultparentrpt=='', 'ultparentrpt'] = pd.NA
        df.ultparentrpt = df.ultparentrpt.astype('Int32')
    elif tbl=='pre':
        df.report = df.report.astype('int32')
        df.line = df.line.astype('int32')
        df.inpth = df.inpth.astype('bool')
        df.negating = df.negating.astype('bool')
    elif tbl=='cal':
        df.grp = df.grp.astype('int16')
        df.arc = df.arc.astype('int16')
        df.negative = df.negative.astype('bool')

    # print(df.dtypes)
    # display(df.head())

    return df

In [107]:
dera_tables = ['sub', 'tag', 'dim', 'num', 'txt', 'ren', 'pre', 'cal']

if False:
    for tbl in dera_tables:
        qres = engine.execute(f"drop table if exists {ingest_schema}.dera_notes_{tbl}")
        print(qres.fetchall())

In [108]:
import io
import zipfile
import datetime

objects=source_bucket.objects.filter(Prefix='SEC-DERA-NOTES/2021_01_notes')

dera_tag = None

for obj in objects:
    if obj.key.endswith('.zip'):
        zipfile_src = s3_source.Object(os.environ['S3_LANDING_BUCKET'],obj.key)
        tmpname = obj.key.split('/')[-1]
        if False:
            zipfile_src.download_file(f'/tmp/{tmpname}')
        zipfile_obj = zipfile.ZipFile(f'/tmp/{tmpname}', mode='r')
        fy_qtr = tmpname.split('.')[0]
        for tbl in dera_tables:
            print(f'{fy_qtr} - {tbl}')
            with zipfile_obj.open(f"{tbl}.tsv") as zf:
                # Read data from ZF into a dataframe.
                dera_df[tbl] = read_dera_notes_table (zf, fy_qtr, tbl)
                if tbl == 'tag':
                    dera_tag = pd.concat([dera_tag, dera_df[tbl]]).drop_duplicates()
        zipfile_obj.close()

        # For now we have all num and sub
        continue

        if False:
            # Alas, there is some minor post-fixing we need to do before ingesting into parquet
            df = dera_df['num']
            num_fields = df.columns
            df = df[df.tag=='ComputedTreasuryFloat']
            treasury_df = dera_df['sub'].loc[dera_df['sub'].fp=='FY', ['adsh', 'cik', 'name','fye', 'fy', ]].merge(df, on='adsh')
            if (len(treasury_df)>0):
                display(treasury_df)
                grouped_df = treasury_df.groupby('cik')
                for key, item in grouped_df:
                    if len(item)==1:
                        df = item[num_fields].copy()
                        df.ddate = pd.to_datetime(f"{item.fy.squeeze().year}1231", format='%Y%m%d')
                        df.version = item.adsh.squeeze()
                        # df = df.astype(dera_df['num'].dtypes.to_dict())
                        print("adding fact (1)")
                        print(df)
                        df = df.astype(dera_df['num'].dtypes.to_dict())
                        dera_df['num'] = dera_df['num'].append(df)
                    else:
                        item = item.sort_values('ddate', ascending=False).reset_index()
                        df = generate_intermediate_ddate_value(item.iloc[0:2])
                        dera_df['num'] = dera_df['num'].append(df)
        if False:
            # Now output as parquet files
            for tbl in dera_tables:
                buf = io.BytesIO()
                dera_df[tbl].to_parquet(path=buf)
                buf.seek(0)
                fy_qtr = dera_df[tbl].iloc[0].srcdir
                trino_bucket.upload_fileobj(Fileobj=buf,
                                            Key=f'trino/{ingest_schema}/{tbl}/{fy_qtr}.parquet')
        else:
            # Import into Trino the slow way...but not tag
            for tbl in dera_tables:
                if tbl == 'tag':
                    continue
                ingest_table = f"dera_notes_{tbl}"
                columnschema = osc.create_table_schema_pairs(dera_df[tbl],
                                                             typemap={"int16":"integer", "Int16":"integer",
                                                                      "datetime64[ns, UTC]":"timestamp(6)"})
                tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    partitioning = array['srcdir'],
    format = 'ORC'
)
"""
                print(tabledef)
                
                continue
                
                qres = engine.execute(tabledef)
                print(qres.fetchall())
                batch_size = {'num':1500,'sub':1500,'tag':1000}[tbl]
                dera_df[tbl].to_sql(ingest_table,
                                    con=engine, schema=ingest_schema, if_exists='append',
                                    index=False,
                                    method=osc.TrinoBatchInsert(batch_size=batch_size, verbose = False))

ingest_table = f"dera_notes_tag"
columnschema = osc.create_table_schema_pairs(dera_tag,
                                             typemap={"int16":"integer", "Int16":"integer",
                                                      "datetime64[ns, UTC]":"timestamp(6)"})
tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
partitioning = array['bucket(version,10)'],
format = 'ORC'
)
"""
print(tabledef)

if False:
    qres = engine.execute(tabledef)
    print(qres.fetchall())

2021_01_notes - sub
2021_01_notes - tag
2021_01_notes - dim
2021_01_notes - num
Inferring floats: start len(df) = 364714
len(df[df.fp=='FY']) = 94933
len(df_has_float) = 4496
len(df_needs_float) = 186692
0001017386-21-000009: merge failed (2)
len(df_prices) = 10
len(df_shares) = 1


,adsh,tag,version,ddate,qtrs,uom,dimh,iprx,value,footnote,footlen,dimn,coreg,durp,datp,dcml,srcdir,fy,fp
15304,0001017386-21-000009,SaleOfStockPricePerShare,us-gaap/2019,2021-09-30 00:00:00+00:00,0,USD,0x00000000,0,4.25,<NA>,0,0,<NA>,0.0,0.0,32767,2021_01_notes,NaT,NaN
15305,0001017386-21-000009,SaleOfStockPricePerShare,us-gaap/2019,2021-09-30 00:00:00+00:00,0,USD,0x00000000,1,4.25,<NA>,0,0,<NA>,0.0,1.0,32767,2021_01_notes,NaT,NaN
15306,0001017386-21-000009,SaleOfStockPricePerShare,us-gaap/2019,2021-09-30 00:00:00+00:00,0,USD,0x00000000,2,4.25,<NA>,0,0,<NA>,0.0,-7.0,32767,2021_01_notes,NaT,NaN
15307,0001017386-21-000009,SaleOfStockPricePerShare,us-gaap/2019,2021-09-30 00:00:00+00:00,0,USD,0x00000000,3,4.25,<NA>,0,0,<NA>,0.0,10.0,32767,2021_01_notes,NaT,NaN
293085,0001017386-21-000009,SharePrice,us-gaap/2019,2017-09-30 00:00:00+00:00,0,USD,0x181642e2b719e89d287174e3abbeab18,0,1.0,<NA>,0,1,<NA>,0.0,5.0,32767,2021_01_notes,NaT,NaN
293086,0001017386-21-000009,SharePrice,us-gaap/2019,2017-10-31 00:00:00+00:00,0,USD,0x181642e2b719e89d287174e3abbeab18,0,0.0875,<NA>,0,1,<NA>,0.0,-3.0,32767,2021_01_notes,NaT,NaN
293087,0001017386-21-000009,SharePrice,us-gaap/2019,2018-01-31 00:00:00+00:00,0,USD,0x181642e2b719e89d287174e3abbeab18,0,0.75,<NA>,0,1,<NA>,0.0,5.0,32767,2021_01_notes,NaT,NaN
293088,0001017386-21-000009,SharePrice,us-gaap/2019,2018-07-31 00:00:00+00:00,0,USD,0x181642e2b719e89d287174e3abbeab18,0,0.875,<NA>,0,1,<NA>,0.0,0.0,32767,2021_01_notes,NaT,NaN
293089,0001017386-21-000009,SharePrice,us-gaap/2019,2020-05-31 00:00:00+00:00,0,USD,0x181642e2b719e89d287174e3abbeab18,0,1.5,<NA>,0,1,<NA>,0.0,0.0,32767,2021_01_notes,NaT,NaN
293090,0001017386-21-000009,SharePrice,us-gaap/2019,2020-11-30 00:00:00+00:00,0,USD,0x30f674b92a037ee8702e5a55dd776b08,0,0.54,<NA>,0,1,<NA>,0.0,0.0,32767,2021_01_notes,NaT,NaN


,adsh,tag,version,ddate,qtrs,uom,dimh,iprx,value,footnote,footlen,dimn,coreg,durp,datp,dcml,srcdir,fy,fp
30945,0001017386-21-000009,EntityCommonStockSharesOutstanding,dei/2019,2020-12-31 00:00:00+00:00,0,shares,0x00000000,0,20728833.0,<NA>,0,0,<NA>,0.0,-14.0,32767,2021_01_notes,NaT,NaN


0001104659-21-000366: merge failed (2)
len(df_prices) = 6
len(df_shares) = 2


/var/folders/lf/r0g8y7yj4rn56tq1_5g2hln00000gq/T/ipykernel_27912/1879381535.py:155: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df_float)
/var/folders/lf/r0g8y7yj4rn56tq1_5g2hln00000gq/T/ipykernel_27912/1879381535.py:155: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df_float)
/var/folders/lf/r0g8y7yj4rn56tq1_5g2hln00000gq/T/ipykernel_27912/1879381535.py:155: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df_float)


...

/var/folders/lf/r0g8y7yj4rn56tq1_5g2hln00000gq/T/ipykernel_27912/1879381535.py:155: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df_float)


,adsh,tag,version,ddate,qtrs,uom,dimh,iprx,value,footnote,footlen,dimn,coreg,durp,datp,dcml,srcdir,fy,fp
117427,0001683168-21-000268,SharePrice,us-gaap/2020,2019-06-30 00:00:00+00:00,0,USD,0x7a23a4a2a9e040b3f23e263592309d01,0,0.3,<NA>,0,1,<NA>,0.0,11.0,32767,2021_01_notes,NaT,NaN
117428,0001683168-21-000268,SharePrice,us-gaap/2020,2018-09-30 00:00:00+00:00,0,USD,0x75d19d7ecae0be6231e6714d8ebf8441,0,1.0,<NA>,0,1,<NA>,0.0,6.0,32767,2021_01_notes,NaT,NaN
117429,0001683168-21-000268,SharePrice,us-gaap/2020,2018-11-30 00:00:00+00:00,0,USD,0x6753807b04e559270006f58ad511fcdf,0,1.0,<NA>,0,1,<NA>,0.0,-12.0,32767,2021_01_notes,NaT,NaN


,adsh,tag,version,ddate,qtrs,uom,dimh,iprx,value,footnote,footlen,dimn,coreg,durp,datp,dcml,srcdir,fy,fp
84884,0001683168-21-000268,CommonStockSharesOutstanding,us-gaap/2020,2019-08-31 00:00:00+00:00,0,shares,0x00000000,0,134498775.0,<NA>,0,0,<NA>,0.0,0.0,32767,2021_01_notes,NaT,NaN
84885,0001683168-21-000268,CommonStockSharesOutstanding,us-gaap/2020,2020-08-31 00:00:00+00:00,0,shares,0x00000000,0,135080441.0,<NA>,0,0,<NA>,0.0,0.0,32767,2021_01_notes,NaT,NaN
84886,0001683168-21-000268,CommonStockSharesOutstanding,us-gaap/2020,2020-11-30 00:00:00+00:00,0,shares,0x00000000,0,135237584.0,<NA>,0,0,<NA>,0.0,0.0,32767,2021_01_notes,NaT,NaN


/var/folders/lf/r0g8y7yj4rn56tq1_5g2hln00000gq/T/ipykernel_27912/1879381535.py:155: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df_float)


75 floats inferred; 0 treasury-based; 75 market-based
2021_01_notes - txt
2021_01_notes - ren
2021_01_notes - pre
2021_01_notes - cal

create table if not exists osc_datacommons_dev.sandbox.dera_notes_tag(
    tag varchar,
    version varchar,
    custom boolean,
    abstract boolean,
    datatype varchar,
    iord varchar,
    crdr varchar,
    tlabel varchar,
    doc varchar,
    srcdir varchar
) with (
partitioning = array['bucket(version,10)'],
format = 'ORC'
)



In [102]:
for tbl in dera_tables:
    display(dera_df[tbl].dtypes)

adsh                        string
cik                          int64
name                        string
LEI                         string
sic                          Int16
countryba                   string
stprba                      string
cityba                      string
zipba                       string
bas1                        string
bas2                        string
baph                        string
countryma                   string
stprma                      string
cityma                      string
zipma                       string
mas1                        string
mas2                        string
countryinc                  string
stprinc                     string
ein                          Int64
former                      string
changed                     string
afs                         string
wksi                          bool
fye                         string
form                        string
period         datetime64[ns, UTC]
fy             datet

tag         string
version     string
custom        bool
abstract      bool
datatype    string
iord        string
crdr        string
tlabel      string
doc         string
srcdir      string
dtype: object

dimh        string
segments    string
segt          bool
srcdir      string
dtype: object

KeyError: 'num'

In [ ]:
# Borrow metadata code from DERA-iceberg if/when we need it

In [ ]:
tablenames = ['ticker'] + [f"dera_notes_{t}" for t in dera_tables]
l = []
for tbl in tablenames:
    qres = engine.execute(f'select count (*) from {ingest_catalog}.{ingest_schema}.{tbl}')
    l.append(qres.fetchall()[0])
print(list(zip(tablenames, l)))

[('ticker', (12421,)), ('dera_sub', (255305,)), ('dera_num', (20197249,)), ('dera_tag', (1897407,))]


In [57]:
dera_df['sub'].loc[dera_df['sub'].name.str.lower().str.startswith('apple')&(dera_df['sub'].form=='10-Q'), ['adsh','fp','period']]

,adsh,form,fp,period,pubfloatusd
2950,0000320193-21-000010,10-Q,Q1,2020-12-31 00:00:00+00:00,<NA>


In [58]:
dera_df['pre'].loc[(dera_df['pre'].adsh=='0000320193-21-000010')&(dera_df['pre'].stmt=='IS')]

,adsh,report,line,stmt,inpth,tag,version,prole,plabel,negating,srcdir
66543,0000320193-21-000010,2,18,IS,True,NetIncomeLoss,us-gaap/2020,totalLabel,Net income,True,2021_01_notes
84450,0000320193-21-000010,2,9,IS,True,GrossProfit,us-gaap/2020,totalLabel,Gross margin,True,2021_01_notes
90789,0000320193-21-000010,2,15,IS,True,NonoperatingIncomeExpense,us-gaap/2020,terseLabel,"Other income/(expense), net",True,2021_01_notes
95605,0000320193-21-000010,2,23,IS,True,WeightedAverageNumberOfSharesOutstandingBasic,us-gaap/2020,terseLabel,Basic (in shares),True,2021_01_notes
97180,0000320193-21-000010,2,19,IS,True,EarningsPerShareAbstract,us-gaap/2020,terseLabel,Earnings per share:,True,2021_01_notes
101858,0000320193-21-000010,2,16,IS,True,IncomeLossFromContinuingOperationsBeforeIncome...,us-gaap/2020,totalLabel,Income before provision for income taxes,True,2021_01_notes
108618,0000320193-21-000010,2,21,IS,True,EarningsPerShareDiluted,us-gaap/2020,terseLabel,Diluted (in dollars per share),True,2021_01_notes
128676,0000320193-21-000010,2,1,IS,True,StatementTable,us-gaap/2020,terseLabel,Statement [Table],True,2021_01_notes
130540,0000320193-21-000010,2,13,IS,True,OperatingExpenses,us-gaap/2020,totalLabel,Total operating expenses,True,2021_01_notes
132391,0000320193-21-000010,2,10,IS,True,OperatingExpensesAbstract,us-gaap/2020,terseLabel,Operating expenses:,True,2021_01_notes


In [109]:
dera_df['num'].loc[dera_df['num'].adsh.str.contains('0000320193-21-000010')].merge(dera_df['dim'].loc[dera_df['dim'].segments.str.contains('ProductOrService')],on='dimh')

,adsh,tag,version,ddate,qtrs,uom,dimh,iprx,value,footnote,footlen,dimn,coreg,durp,datp,dcml,srcdir_x,segments,segt,srcdir_y
0,0000320193-21-000010,RevenueFromContractWithCustomerExcludingAssess...,us-gaap/2020,2019-12-31 00:00:00+00:00,1,USD,0x94c1611529456c7cf3987f92ddf02327,0,79104000000.0,<NA>,0,1,<NA>,0.013699,3.0,-6,2021_01_notes,ProductOrService=Product;,True,2021_01_notes
1,0000320193-21-000010,RevenueFromContractWithCustomerExcludingAssess...,us-gaap/2020,2020-12-31 00:00:00+00:00,1,USD,0x94c1611529456c7cf3987f92ddf02327,0,95678000000.0,<NA>,0,1,<NA>,0.013699,5.0,-6,2021_01_notes,ProductOrService=Product;,True,2021_01_notes
2,0000320193-21-000010,CostOfGoodsAndServicesSold,us-gaap/2020,2019-12-31 00:00:00+00:00,1,USD,0x94c1611529456c7cf3987f92ddf02327,0,52075000000.0,<NA>,0,1,<NA>,0.013699,3.0,-6,2021_01_notes,ProductOrService=Product;,True,2021_01_notes
3,0000320193-21-000010,CostOfGoodsAndServicesSold,us-gaap/2020,2020-12-31 00:00:00+00:00,1,USD,0x94c1611529456c7cf3987f92ddf02327,0,62130000000.0,<NA>,0,1,<NA>,0.013699,5.0,-6,2021_01_notes,ProductOrService=Product;,True,2021_01_notes
4,0000320193-21-000010,RevenueFromContractWithCustomerExcludingAssess...,us-gaap/2020,2019-12-31 00:00:00+00:00,1,USD,0x58c22680ab8dbbfb662ff4e14055c1bd,0,12715000000.0,<NA>,0,1,<NA>,0.013699,3.0,-6,2021_01_notes,ProductOrService=Service;,True,2021_01_notes
5,0000320193-21-000010,RevenueFromContractWithCustomerExcludingAssess...,us-gaap/2020,2019-12-31 00:00:00+00:00,1,USD,0x58c22680ab8dbbfb662ff4e14055c1bd,1,12715000000.0,<NA>,0,1,<NA>,0.013699,3.0,-6,2021_01_notes,ProductOrService=Service;,True,2021_01_notes
6,0000320193-21-000010,RevenueFromContractWithCustomerExcludingAssess...,us-gaap/2020,2020-12-31 00:00:00+00:00,1,USD,0x58c22680ab8dbbfb662ff4e14055c1bd,0,15761000000.0,<NA>,0,1,<NA>,0.013699,5.0,-6,2021_01_notes,ProductOrService=Service;,True,2021_01_notes
7,0000320193-21-000010,RevenueFromContractWithCustomerExcludingAssess...,us-gaap/2020,2020-12-31 00:00:00+00:00,1,USD,0x58c22680ab8dbbfb662ff4e14055c1bd,1,15761000000.0,<NA>,0,1,<NA>,0.013699,5.0,-6,2021_01_notes,ProductOrService=Service;,True,2021_01_notes
8,0000320193-21-000010,CostOfGoodsAndServicesSold,us-gaap/2020,2019-12-31 00:00:00+00:00,1,USD,0x58c22680ab8dbbfb662ff4e14055c1bd,0,4527000000.0,<NA>,0,1,<NA>,0.013699,3.0,-6,2021_01_notes,ProductOrService=Service;,True,2021_01_notes
9,0000320193-21-000010,CostOfGoodsAndServicesSold,us-gaap/2020,2020-12-31 00:00:00+00:00,1,USD,0x58c22680ab8dbbfb662ff4e14055c1bd,0,4981000000.0,<NA>,0,1,<NA>,0.013699,5.0,-6,2021_01_notes,ProductOrService=Service;,True,2021_01_notes


In [110]:
dera_df['dim'].loc[dera_df['dim'].segments.str.contains('ProductOrService')]

,dimh,segments,segt,srcdir
825,0xf6f338f14df0ab9c8d99ef70a5757365,ProductOrService=OtherServices;,True,2021_01_notes
943,0x34775e161ba8aa1459e13b6067718561,ProductOrService=ConstructionMaterials;,True,2021_01_notes
946,0xdcaf6cb2ecc7be1dbdda70b501422b97,ProductOrService=SteelProducts;,True,2021_01_notes
1023,0xd7bad0a518425894d3997819cb5a1473,ProductOrService=Hardlines;,True,2021_01_notes
1138,0x14db04e2d529f59b52661db82cddf977,ProductOrService=Consumables;,True,2021_01_notes
...,...,...,...,...
26309,0xfc176308f430bb7cb04b6791eb7dced2,ConcentrationRiskByType=ChinaCustomerConcentra...,True,2021_01_notes
26330,0x217863f2186cf606a71cc04ea58a31fc,ConsolidatedEntities=GarrettWalkerHomes;LegalE...,True,2021_01_notes
26331,0x11227cddb6cd8b34808d2c7ff9cdb7a1,LegalEntity=LandseaHomes;ProductOrService=Home...,True,2021_01_notes
26344,0x74a00d60c0c1ee1ff24664ff8eb8673e,LegalEntity=LandseaHomes;ProductOrService=LotS...,True,2021_01_notes


In [111]:
dera_df['num'].loc[dera_df['num'].adsh.str.contains('0000320193-21-000010')]

,adsh,tag,version,ddate,qtrs,uom,dimh,iprx,value,footnote,footlen,dimn,coreg,durp,datp,dcml,srcdir
31354,0000320193-21-000010,NetIncomeLoss,us-gaap/2020,2020-12-31 00:00:00+00:00,1,USD,0xaff49d53012fa8fe16e4b169dde56fd7,0,28755000000.0,<NA>,0,1,<NA>,0.013699,5.0,-6,2021_01_notes
31355,0000320193-21-000010,NetIncomeLoss,us-gaap/2020,2019-12-31 00:00:00+00:00,1,USD,0xaff49d53012fa8fe16e4b169dde56fd7,0,22236000000.0,<NA>,0,1,<NA>,0.013699,3.0,-6,2021_01_notes
35927,0000320193-21-000010,NetIncomeLoss,us-gaap/2020,2020-12-31 00:00:00+00:00,1,USD,0x00000000,3,28755000000.0,<NA>,0,0,<NA>,0.013699,5.0,-6,2021_01_notes
35928,0000320193-21-000010,NetIncomeLoss,us-gaap/2020,2020-12-31 00:00:00+00:00,1,USD,0x00000000,2,28755000000.0,<NA>,0,0,<NA>,0.013699,5.0,-6,2021_01_notes
35929,0000320193-21-000010,NetIncomeLoss,us-gaap/2020,2020-12-31 00:00:00+00:00,1,USD,0x00000000,1,28755000000.0,<NA>,0,0,<NA>,0.013699,5.0,-6,2021_01_notes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341397,0000320193-21-000010,OtherComprehensiveIncomeLossTaxPortionAttribut...,us-gaap/2020,2020-12-31 00:00:00+00:00,1,USD,0x516d219075d54185cb473ea3924371fc,0,-83000000.0,<NA>,0,1,<NA>,0.013699,5.0,-6,2021_01_notes
341952,0000320193-21-000010,DebtInstrumentInterestRateStatedPercentage,us-gaap/2020,2020-12-31 00:00:00+00:00,0,pure,0x4b3645b38476a5a0ba7ceec6107e7d49,0,0.0,<NA>,0,3,<NA>,0.0,5.0,32767,2021_01_notes
341953,0000320193-21-000010,DebtInstrumentInterestRateStatedPercentage,us-gaap/2020,2020-12-31 00:00:00+00:00,0,pure,0x9a57e58833488221b90092c0022bc714,0,0.0465,<NA>,0,3,<NA>,0.0,5.0,32767,2021_01_notes
344569,0000320193-21-000010,Revenues,us-gaap/2020,2020-12-31 00:00:00+00:00,1,USD,0xb8ba6a9ef479afc51e87d63625c7949d,0,-104000000.0,<NA>,0,3,<NA>,0.013699,5.0,-6,2021_01_notes


In [78]:
dera_df['num'].size

1613861